<a href="https://colab.research.google.com/github/mounikamarreddy/BERTFinetuning-NLPTasks/blob/main/GPT_2_French_Probabilities.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 120 kB 45.7 MB/s 
     |████████████████████████████████| 6.6 MB 38.4 MB/s 


In [2]:
!wget https://raw.githubusercontent.com/icml2019paper2428/Towards-A-Deep-and-Unified-Understanding-of-Deep-Neural-Models-in-NLP/master/Interpreter.py

--2022-09-24 22:45:39--  https://raw.githubusercontent.com/icml2019paper2428/Towards-A-Deep-and-Unified-Understanding-of-Deep-Neural-Models-in-NLP/master/Interpreter.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6692 (6.5K) [text/plain]
Saving to: ‘Interpreter.py’

Interpreter.py      100%[===================>]   6.54K  --.-KB/s    in 0s      

2022-09-24 22:45:39 (46.1 MB/s) - ‘Interpreter.py’ saved [6692/6692]



In [3]:
!ls

Interpreter.py	sample_data


In [115]:
# import torch
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
# from Interpreter import Interpreter 
 
# def Phi(x):
#     global model
#     result = model(inputs_embeds=x)[0]
#     return result # return the logit of last word
 
# model_path = "dbddv01/gpt2-french-small"
# model = GPT2LMHeadModel.from_pretrained(model_path, output_attentions=True)
# tokenizer = GPT2Tokenizer.from_pretrained(model_path)
 
# input_embedding_weight_std = (
#     model.get_input_embeddings().weight.view(1,-1)
#     .std().item()
# )
 
# text = "Les deux garcons partent en courant de l'école pour arriver les premiers au match de foot"
# inputs = tokenizer.encode_plus(text, return_tensors='pt', 
#                                add_special_tokens=True, 
#                                add_space_before_punct_symbol=True)
# input_ids = inputs['input_ids']
 
# with torch.no_grad():
#     x = model.get_input_embeddings()(input_ids).squeeze()

 
# interpreter = Interpreter(x=x, Phi=Phi, 
#                           scale=10*input_embedding_weight_std,
#                           words=text.split(' ')).to(model.device)
 
# # This will take sometime.
# interpreter.optimize(iteration=1000, lr=0.01, show_progress=True)
# interpreter.get_sigma()
# interpreter.visualize()

In [84]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import pandas as pd
import numpy as np
import torch

In [79]:
from google.colab import files
uploaded = files.upload()

Saving Ordre_Stims_comprehension.xlsx to Ordre_Stims_comprehension.xlsx


In [82]:
data = pd.read_excel("Ordre_Stims_comprehension.xlsx")
print(data.head())

                                              Phrase  ordre stim  nb_mots  \
0  lundi mardi mercredi jeudi vendredi samedi dim...           1        7   
1  Les deux garcons partent en courant de l'école...           2       16   
2  janvier fevrier mars avril mai juin juillet ao...           3        9   
3  cinq heures six heures sept heures huit heures...           4       12   
4  La maman de Nicolas lui explique que trois vit...           5       16   

   temps début     durée  
0            0  4.014512  
1           14  4.697188  
2           28  4.745420  
3           42  3.903061  
4           56  5.004036  


In [83]:
model_path = "dbddv01/gpt2-french-small"
model_path = "antoiloui/belgpt2"
model = GPT2LMHeadModel.from_pretrained(model_path, output_attentions=True)
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
 
input_embedding_weight_std = (
    model.get_input_embeddings().weight.view(1,-1)
    .std().item()
)

In [103]:
def generate_probs(text, input_ids,bb):
  probs = []
  actual_tokens = text.split(' ')
  ati, bti, a2b = 0, 0, 0
  while ati < len(actual_tokens):
    atok = actual_tokens[ati]
    toki = 0
    temp = []
    while toki < len(atok):
      btok = tokenizer.decode(input_ids[0][a2b])
      temp.append(bb[a2b][input_ids[0][a2b]])
      toki += len(btok)
      if btok.startswith(' '):
        toki = toki-1
      a2b+=1
      if (toki == len(atok)):
        break
    ati+=1
    #print(len(temp))
    if len(temp)>1:
      temp = np.sum(np.array(temp))
      temp = [temp]
    probs.append(temp[0])
  return probs

In [105]:
final_prob = []
#text = "Les deux garcons partent en courant de l'école pour arriver les premiers au match de foot"
for text in data.Phrase:
  inputs = tokenizer.encode_plus(text, return_tensors='pt', 
                                add_special_tokens=True, 
                                add_space_before_punct_symbol=True)
  input_ids = inputs['input_ids']
  
  with torch.no_grad():
      x = model.get_input_embeddings()(input_ids).squeeze()

  result = model(inputs_embeds=x)[0]
  result = torch.nn.functional.softmax(result)
  final = result.detach().numpy()
  probs = generate_probs(text, input_ids, final)
  final_prob.append(probs)

Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True} not recognized.
Keyword arguments {'add_space_before_punct_symbol': True

In [109]:
df = pd.DataFrame(final_prob)
df.shape

(26, 19)

In [112]:
finaldf = pd.concat([data,df], axis=1)
finaldf.shape

(26, 24)

In [113]:
finaldf.to_excel("Ordre_Stims_comprehension_probabilities.xlsx", index=None)

In [114]:
files.download('Ordre_Stims_comprehension_probabilities.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>